In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
from training.training import finetune
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from filtering import compute_perplexity, filter_dataset
import numpy as np

In [4]:
disk_path = "data/train_eval_test_split"

# code_exercises = load_dataset("jinaai/code_exercises")["train"].select(range(25000))
# train_test_split = code_exercises.train_test_split(test_size=0.1)
# train_eval_split = train_test_split["train"].train_test_split(test_size=1/9)

# train_dataset = train_eval_split["train"]
# eval_dataset = train_eval_split["test"]
# test_dataset = train_test_split["test"]

# DatasetDict({
#     "train": train_dataset,
#     "eval": eval_dataset,
#     "test": test_dataset,
# }).save_to_disk(disk_path)

train_eval_test_split = load_from_disk(disk_path)

train_eval_split = DatasetDict({
    "train": train_eval_test_split["train"],
    "test": train_eval_test_split["eval"]
})

test_dataset = train_eval_test_split["test"]

In [3]:
model_name = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.__dict__.get("pad_token") is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [4]:
finetune_kwargs = {
    "per_device_train_batch_size": 16,
    "warmup_ratio": 0.1,
    "per_device_eval_batch_size": 16,
    "gradient_accumulation_steps": 8,
    "evaluation_strategy": "steps",
    "eval_steps": 40,
    "save_strategy": "steps",
    "save_steps": 40,
    "num_train_epochs": 1,
}

In [7]:
finetuned_model = finetune(
    model,
    tokenizer,
    train_eval_split,
    run_name = "full_dataset_finetuning",
    **finetune_kwargs,
)

/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Model Preparation Time
40,0.138600,0.134753,0.002800
80,0.131700,0.126978,0.002800
120,0.120800,0.122841,0.002800


eval/loss,█▁▁▁
eval/model_preparation_time,▁▁▁▁
eval/runtime,█▁▂▁
eval/samples_per_second,▁█▇█
eval/steps_per_second,▁█▇█
train/epoch,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
train/grad_norm,██▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▂▃▃▄▇████▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.12284


In [5]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.__dict__.get("pad_token") is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
finetune_kwargs = {
    "per_device_train_batch_size": 8,
    "warmup_ratio": 0.1,
    "per_device_eval_batch_size": 8,
    "gradient_accumulation_steps": 16,
    "evaluation_strategy": "steps",
    "eval_steps": 40,
    "save_strategy": "steps",
    "save_steps": 40,
    "num_train_epochs": 1,
    "output_dir": "deepseek",
}

In [7]:
finetuned_model = finetune(
    model,
    tokenizer,
    train_eval_split,
    run_name = "full_dataset_finetuning",
    **finetune_kwargs,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: antonii-belyshev. Use `wandb login --relogin` to force relogin


/home/coder/.local/lib/python310-conda-cuda-torch21/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Model Preparation Time
40,0.295100,0.286347,0.004400
80,0.264500,0.273741,0.004400
120,0.262700,0.266788,0.004400


eval/loss,█▁▁▁
eval/model_preparation_time,▁▁▁▁
eval/runtime,▁▇█▇
eval/samples_per_second,▁▁▁▁
eval/steps_per_second,▁▁▁▁
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▄▄▅███▇▇▇▇▇▇▆▆▆▆▆▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▁▁
train/loss,▇▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,0.26679
